# DynamoDB Basics Demo
---

**Setup Imports**

In [1]:
import json
import decouple
from repository.sample_txn_repo import SampleTxnRepo

config = decouple.AutoConfig(' ')
repo = SampleTxnRepo(config)

**Scan**

output scanned items from table as string

In [2]:
scan_result = repo.scan()
str(json.dumps(scan_result))

'{"transactions": [{"txn_id": "2", "client_id": "1", "amount": 100.0, "ccy": "USD", "balance": {"PHP": 100, "USD": 100}, "business_date": "2022"}, {"txn_id": "1", "client_id": "1", "amount": 100.0, "ccy": "PHP", "balance": {"PHP": 100, "USD": 0}, "business_date": "2021"}, {"txn_id": "4", "client_id": "2", "amount": 200.0, "ccy": "USD", "balance": {"PHP": 200, "USD": 200}, "business_date": "2024"}, {"txn_id": "3", "client_id": "2", "amount": 200.0, "ccy": "PHP", "balance": {"PHP": 200, "USD": 0}, "business_date": "2023"}]}'

convert to pandas dataframe:

In [3]:
df_pandas = repo.convert_list_of_dicts_to_pandas_df(scan_result["transactions"])
df_pandas

,txn_id,client_id,amount,ccy,balance,business_date
0,2,1,100.0,USD,"{'PHP': 100, 'USD': 100}",2022
1,1,1,100.0,PHP,"{'PHP': 100, 'USD': 0}",2021
2,4,2,200.0,USD,"{'PHP': 200, 'USD': 200}",2024
3,3,2,200.0,PHP,"{'PHP': 200, 'USD': 0}",2023


convert to spark dataframe:

In [4]:
df_spark = repo.convert_pandas_df_to_spark_df(df_pandas)
df_spark.show()

+------+---------+------+---+--------------------+-------------+
|txn_id|client_id|amount|ccy|             balance|business_date|
+------+---------+------+---+--------------------+-------------+
|     2|        1| 100.0|USD|{'PHP': 100, 'USD...|         2022|
|     1|        1| 100.0|PHP|{'PHP': 100, 'USD...|         2021|
|     4|        2| 200.0|USD|{'PHP': 200, 'USD...|         2024|
|     3|        2| 200.0|PHP|{'PHP': 200, 'USD...|         2023|
+------+---------+------+---+--------------------+-------------+



**Get Item**

In [5]:
txn_id_1 = repo.get_item("1")
txn_id_1

OrderedDict([('txn_id', '1'),
             ('client_id', '1'),
             ('amount', 100.0),
             ('ccy', 'PHP'),
             ('balance', {'PHP': Decimal('100'), 'USD': Decimal('0')}),
             ('business_date', '2021')])

In [6]:
df_pandas = repo.convert_list_of_dicts_to_pandas_df([txn_id_1])
df_pandas

,txn_id,client_id,amount,ccy,balance,business_date
0,1,1,100.0,PHP,"{'PHP': Decimal('100'), 'USD': Decimal('0')}",2021


**Put Item**

with example of new item inserted with different schema

by default put item operation always overwrites the currect item based on its primary key

with conditional operations it can check whether item exists before executing the put item operation

In [7]:
item_to_put = {
    "txn_id": "5",
    "client_id": "3",
    "amount": 100,
    "ccy": "PHP",
    "balance": {
        "PHP": 100,
        "USD": 0
    },
    "business_date": "2025",
    "some_random_new_field": "xxx"
}

In [8]:
repo.put_item(item_to_put)

[2023-05-07 15:59:11][sample_txn_repo][put_item][DEBUG]: Saved: {'txn_id': '5', 'client_id': '3', 'amount': 100, 'ccy': 'PHP', 'balance': {'PHP': 100, 'USD': 0}, 'business_date': '2025', 'some_random_new_field': 'xxx'}


In [9]:
txn_id_5 = repo.get_item("5")
txn_id_5

OrderedDict([('txn_id', '5'),
             ('client_id', '3'),
             ('amount', 100.0),
             ('ccy', 'PHP'),
             ('balance', {'PHP': Decimal('100'), 'USD': Decimal('0')}),
             ('business_date', '2025')])

**Delete Item**

In [10]:
repo.delete_item("5")

[2023-05-07 15:59:11][sample_txn_repo][delete_item][DEBUG]: Deleted item with key: {'txn_id': '5'}


**Query with GSI**

In [11]:
txn_hist_client_1, last_key = repo.query("1", "2020", "2030")
print("data: " + str(txn_hist_client_1))
print("last key: " + str(last_key))

data: OrderedDict([('transactions', [OrderedDict([('txn_id', '1'), ('client_id', '1'), ('amount', 100.0), ('ccy', 'PHP'), ('balance', {'PHP': Decimal('100'), 'USD': Decimal('0')}), ('business_date', '2021')]), OrderedDict([('txn_id', '2'), ('client_id', '1'), ('amount', 100.0), ('ccy', 'USD'), ('balance', {'PHP': Decimal('100'), 'USD': Decimal('100')}), ('business_date', '2022')])])])
last key: None


In [12]:
df_pandas = repo.convert_list_of_dicts_to_pandas_df(txn_hist_client_1["transactions"])
df_pandas

,txn_id,client_id,amount,ccy,balance,business_date
0,1,1,100.0,PHP,"{'PHP': Decimal('100'), 'USD': Decimal('0')}",2021
1,2,1,100.0,USD,"{'PHP': Decimal('100'), 'USD': Decimal('100')}",2022
